In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# pretrained models
import torchvision
from torchvision import models, transforms

import albumentations as A


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 30 16:35:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# What's in this dataset?
import os
os.listdir('drive/MyDrive/DeepLearningProject/PotholeDataset')



['test', 'train']

In [6]:
!find . -name "*.DS_Store" -type f -delete

In [7]:
# create a dataframe for our data
data_path = 'drive/MyDrive/DeepLearningProject/PotholeDataset'

rows = []
for dataset in ['test','train']:
    for label in os.listdir(data_path + f'/{dataset}'):
        for image in os.listdir(data_path + f'/{dataset}' + f'/{label}'):
            row = dict()
            row['image_file'] = image
            row['label'] = label
            row['dataset'] = dataset
        
            # a bit redudant, could build from other data in __getitem__ if wanted
            row['image_path'] = data_path + f'/{dataset}' + f'/{label}'
            rows.append(row)
        
df = pd.DataFrame(rows)
print(len(df))
df.head()

1351


,image_file,label,dataset,image_path
0,2.jpg,Plain,test,drive/MyDrive/DeepLearningProject/PotholeDatas...
1,7.jpg,Plain,test,drive/MyDrive/DeepLearningProject/PotholeDatas...
2,1.jpg,Plain,test,drive/MyDrive/DeepLearningProject/PotholeDatas...
3,3.jpg,Plain,test,drive/MyDrive/DeepLearningProject/PotholeDatas...
4,6.jpg,Plain,test,drive/MyDrive/DeepLearningProject/PotholeDatas...


In [8]:
len(df)

1351

In [9]:
# training and validation data
df_train = df[df['dataset'] == 'train'].reset_index(drop=True)
df_val = df[df['dataset'] == 'test'].reset_index(drop=True)
len(df_train), len(df_val)

(1316, 35)

In [10]:
# Pull portion of the training for model verification
# df_train = df_train.sample(frac =.80)


In [11]:
%%capture

!pip install opencv-python

## Resizing the image

In [12]:
import cv2

def resize_img(path, size):
    img = cv2.imread(path)
    
    start = time.time()
    try:
        img = cv2.resize(img, size)
        cv2.imwrite(path, img)
    except:
        # look at the image
        print(path)
    end = time.time()
    return end - start

# resize all of the images to 512x512
total_time_resize = 0.0
for idx in tqdm(range(len(df_train))):
    row = df_train.iloc[idx]
    image_path = row['image_path']
    fname = row['image_file']
    path = image_path+'/'+fname
    
    total_time_resize += resize_img(path, (224, 224))
    
for idx in tqdm(range(len(df_val))):
    row = df_val.iloc[idx]
    image_path = row['image_path']
    fname = row['image_file']
    path = image_path+'/'+fname
    
    total_time_resize += resize_img(path, (224, 224))

  0%|          | 0/1316 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

## Data Processing

In [13]:
device = torch.device('cuda')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [14]:
# class PotholeDataset(Dataset):
#     def __init__(self, df):
#         self.df = df
        
#         # label dictionary
#         self.label_dict = {'Plain':0, 'Pothole':1}
    
#     def __len__(self):
#         return len(self.df)

#     def __iter__(self):
#       for b in self.dl:
#         yield to_device(b, self.device)
    
#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
        
#         # get ingredients for retrieving image
#         image_path = row['image_path']
#         fname = row['image_file']
#         path = image_path+'/'+fname
        
#         # read the img
#         img = cv2.imread(path)
        
#         # convert to RGB
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         # move color channels to correct spot
#         img = np.transpose(img, (2, 0, 1))
        
#         # convert to [0,1] scale
#         img = torch.tensor(img / 255.).float()
        
#         label = torch.tensor(self.label_dict[row['label']])
        
#         return img.to(device), label.to(device)

In [15]:
class PotholeDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
        # label dictionary
        self.label_dict = {'Plain':0, 'Pothole':1}

        self.transform = A.Compose([
              A.RandomRotate90(p=.1),
              A.HorizontalFlip(p=0.5),
              A.RandomBrightnessContrast(p=0.2)
          ])
    
    def __len__(self):
        return len(self.df)

    def __iter__(self):
      for b in self.dl:
        yield to_device(b, self.device)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # get ingredients for retrieving image
        image_path = row['image_path']
        fname = row['image_file']
        path = image_path+'/'+fname
        
        # read the img
        img = cv2.imread(path)
        
        # convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # augment
        transformed = self.transform(image=img)
        img = torch.tensor(transformed['image']).float()
        
        # move color channels to correct spot
        img = np.transpose(img, (2, 0, 1))
        
        # convert to [0,1] scale
        img = torch.tensor(img / 255.).float()
        
        label = torch.tensor(self.label_dict[row['label']])
        
        return img.to(device), label.to(device)

In [16]:
ds_train = PotholeDataset(df_train)
dl_train = DataLoader(ds_train, batch_size = 10, shuffle=True)

ds_val = PotholeDataset(df_val)
dl_val = DataLoader(ds_val, batch_size = 10, shuffle=True)

In [17]:
# image_path = df_train['image_path'][100]

# fname = row['image_file']
# path = image_path+'/'+fname
# path = 'drive/MyDrive/DeepLearningProject/PotholeDataset/train/Plain/340.imagef7973d0b-ad11-4bbc-bc81-5d606377259f.jpg'
# # read the img
# img = cv2.imread(path)


In [18]:
# plt.imshow(img)

In [19]:
df_train

,image_file,label,dataset,image_path
0,294.jpg,Pothole,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
1,378.jpg,Pothole,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
2,355.jpg,Pothole,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
3,285.20190319-potholes-ka.jpg,Pothole,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
4,102.jpg,Pothole,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
...,...,...,...,...
1311,284.federation-way-1.jpg,Plain,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
1312,361.300px-MumbaiPuneExpressway.jpg,Plain,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
1313,87.road.jpg,Plain,train,drive/MyDrive/DeepLearningProject/PotholeDatas...
1314,6.country-road-770x439_c.jpg,Plain,train,drive/MyDrive/DeepLearningProject/PotholeDatas...


In [20]:
# for img, label in tqdm(dl_train):
#     None

## Model Training Pipeline

In [21]:
def one_pass(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0

    if backwards:
      for x, y in tqdm(dataloader):
          
          y_pred = model(x)
          loss = lossFun(y_pred, y)
          total_loss += loss.item()
          
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
    else:
        for x, y in dataloader:
          
          y_pred = model(x)
          loss = lossFun(y_pred, y)
          total_loss += loss.item()


    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss

# def one_pass_acc(model, dataloader, num_points):
#     model.eval()
#     total_incorrect = 0
    
#     softmax = nn.LogSoftmax(dim=1)
    
#     for x, y in dataloader:
#         y_pred = softmax(model(x))
#         y_pred = torch.argmax(y_pred, dim=1)
        
#         total_incorrect += torch.count_nonzero(y - y_pred).item()
        
#     percent_wrong = total_incorrect / num_points
#     return 1 - percent_wrong


def one_pass_acc(model, dataloader, num_points):
    model.eval()
    total_incorrect = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    softmax = nn.LogSoftmax(dim=1)

    for x, y in dataloader:
        y_pred = softmax(model(x))
        y_pred = torch.argmax(y_pred, dim=1)
        
        total_incorrect += torch.count_nonzero(y - y_pred).item()
        TP += torch.count_nonzero(y_pred * y)
        TN += torch.count_nonzero((y_pred - 1) * (y - 1))
        FP += torch.count_nonzero(y_pred * (y - 1))
        FN += torch.count_nonzero((y_pred - 1) * y)
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)    
        
    percent_wrong = total_incorrect / num_points
    return 1 - percent_wrong, precision.item(), recall.item()

In [22]:
def model_train_eval(model, dl_train, optimizer, num_epochs=5):

  lossFun = nn.CrossEntropyLoss()
  lossFun = lossFun.to(device)

  train_losses = []
  valid_losses = []

  for epoch in tqdm(range(num_epochs)):
      print('Epoch: ', epoch)
      
      train_loss = one_pass(model, dl_train, optimizer, lossFun)
      train_losses.append(train_loss)
      
      
      valid_loss = one_pass(model, dl_val, optimizer, lossFun, backwards=False)
      valid_losses.append(valid_loss)
      
      
      
      train_acc, train_prec, train_recall = one_pass_acc(model, dl_train, len(ds_train))
      valid_acc, valid_prec, valid_recall = one_pass_acc(model, dl_val,   len(ds_val))
      
      print('Train: ')
      print('- loss: ', round(train_loss, 5))
      print('- Accuracy: ', round(train_acc*100,2), "%")
      print('- Precision: ', round(train_prec*100,2), "%")
      print('- Recall: ', round(train_recall*100,2), "%")
      print()
      print('Valid: ')
      print('- loss: ', round(valid_loss, 5))
      print('- Accuracy: ', round(valid_acc*100,2), "%")
      print('- Precision: ', round(valid_prec*100,2), "%")
      print('- Recall: ', round(valid_recall*100,2), "%")


  return model, train_losses, valid_losses

## Baseline CNN

In [23]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(44944, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x




In [24]:
net = Net()
net.to(device)

optimizer = optim.Adam(net.parameters(), lr = 0.001)
trained_net, train_losses_net, valid_losses_net = model_train_eval(net, dl_train, optimizer, num_epochs=15)


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/132 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train: 
- loss:  0.5491
- Accuracy:  81.53 %
- Precision:  80.92 %
- Recall:  97.59 %

Valid: 
- loss:  0.74731
- Accuracy:  62.86 %
- Precision:  63.64 %
- Recall:  95.45 %
Epoch:  1


...

Train: 
- loss:  0.14299
- Accuracy:  95.82 %
- Precision:  95.0 %
- Recall:  99.48 %

Valid: 
- loss:  0.96023
- Accuracy:  74.29 %
- Precision:  70.97 %
- Recall:  100.0 %
Epoch:  13


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.08382
- Accuracy:  97.72 %
- Precision:  97.63 %
- Recall:  99.27 %

Valid: 
- loss:  0.70841
- Accuracy:  80.0 %
- Precision:  77.78 %
- Recall:  95.45 %
Epoch:  14


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.07819
- Accuracy:  98.48 %
- Precision:  99.37 %
- Recall:  98.54 %

Valid: 
- loss:  0.66679
- Accuracy:  82.86 %
- Precision:  80.77 %
- Recall:  95.45 %


In [25]:
!nvidia-smi


Thu Jun 30 05:26:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    34W /  70W |   1496MiB / 15109MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## CNN With Dropout

In [26]:
import torch.nn as nn
import torch.nn.functional as F
class NetDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.do1 = nn.Dropout(0.2)  # 20% Probability
        self.fc1 = nn.Linear(44944, 120)
        self.do2 = nn.Dropout(0.2)  # 20% Probability
        self.fc2 = nn.Linear(120, 84)
        self.do3 = nn.Dropout(0.1)  # 10% Probability
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.do1(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.do2(x)
        x = F.relu(self.fc2(x))
        x = self.do3(x)
        x = self.fc3(x)
        return x

In [27]:
netdropout = NetDropout()
netdropout.to(device)

optimizer = optim.Adam(netdropout.parameters(), lr = 0.001)
trained_netdropout, train_losses_drop, valid_losses_drop = model_train_eval(netdropout, dl_train, optimizer, num_epochs=15)


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/132 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train: 
- loss:  0.52034
- Accuracy:  87.61 %
- Precision:  89.61 %
- Recall:  93.83 %

Valid: 
- loss:  0.44564
- Accuracy:  80.0 %
- Precision:  77.78 %
- Recall:  95.45 %
Epoch:  1


...

Train: 
- loss:  0.13209
- Accuracy:  97.72 %
- Precision:  98.03 %
- Recall:  98.85 %

Valid: 
- loss:  0.81659
- Accuracy:  85.71 %
- Precision:  81.48 %
- Recall:  100.0 %
Epoch:  13


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.07643
- Accuracy:  98.18 %
- Precision:  98.44 %
- Recall:  99.06 %

Valid: 
- loss:  0.78487
- Accuracy:  82.86 %
- Precision:  78.57 %
- Recall:  100.0 %
Epoch:  14


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.0944
- Accuracy:  98.33 %
- Precision:  98.44 %
- Recall:  99.27 %

Valid: 
- loss:  0.79312
- Accuracy:  85.71 %
- Precision:  81.48 %
- Recall:  100.0 %


## Baseline Pretrained: VGG

In [28]:
# New Model 

# to make this a bit cleaner you can make a new model class
# use model.features1, model.features2, and model.classifier to set the learning rates
class Tune_ResNet(nn.Module):
    def __init__(self):
        super(Tune_ResNet, self).__init__()
        VGG16 = torchvision.models.vgg16(pretrained=True)
        layers = list(VGG16.children())[:1]
        self.features1 = nn.Sequential(*layers[:1])
        self.features2 = nn.Sequential(*layers[1:])
        self.classifier = nn.Linear(25088 , 2)
        self.unroll = nn.Flatten()
    
    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = self.unroll(x)
        x = self.classifier(x)
        return x
    
VGG_Tune = Tune_ResNet()
VGG_Tune.to(device)
# summary(VGG_Tune, input_size = (3, 224, 224), device='cpu')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Tune_ResNet(
  (features1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
    

In [29]:
max_lr = 0.01
params = []
for i, layer in enumerate(VGG_Tune.children()):
    if i < 6:
        params.append({'params': layer.parameters(), 'lr': max_lr / 100})
    elif 5 < i < 9:
        params.append({'params': layer.parameters(), 'lr': max_lr / 10})
    else:
        params.append({'params': layer.parameters()})
        
print(params)


# only the parameters we didn't manually set the learning rate for inherit the learning rate set when defining the optimizer
optimizer = optim.Adam(params, lr = max_lr)

[{'params': <generator object Module.parameters at 0x7f2d822961d0>, 'lr': 0.0001}, {'params': <generator object Module.parameters at 0x7f2d822965d0>, 'lr': 0.0001}, {'params': <generator object Module.parameters at 0x7f2d82296550>, 'lr': 0.0001}, {'params': <generator object Module.parameters at 0x7f2d822964d0>, 'lr': 0.0001}]


In [30]:
trained_vgg_tune, train_losses_vgg_tune, valid_losses_vgg_tune = model_train_eval(VGG_Tune, dl_train, optimizer, num_epochs=15)

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/132 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train: 
- loss:  0.1737
- Accuracy:  92.55 %
- Precision:  99.31 %
- Recall:  90.38 %

Valid: 
- loss:  0.51956
- Accuracy:  85.71 %
- Precision:  90.48 %
- Recall:  86.36 %
Epoch:  1


...

Train: 
- loss:  0.02099
- Accuracy:  96.12 %
- Precision:  94.94 %
- Recall:  100.0 %

Valid: 
- loss:  0.60017
- Accuracy:  88.57 %
- Precision:  84.62 %
- Recall:  100.0 %
Epoch:  13


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.06845
- Accuracy:  93.31 %
- Precision:  100.0 %
- Recall:  90.79 %

Valid: 
- loss:  0.46231
- Accuracy:  82.86 %
- Precision:  94.44 %
- Recall:  77.27 %
Epoch:  14


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.07398
- Accuracy:  91.64 %
- Precision:  89.68 %
- Recall:  100.0 %

Valid: 
- loss:  2.9411
- Accuracy:  77.14 %
- Precision:  73.33 %
- Recall:  100.0 %


## AlexNet

In [32]:
import torch.nn as nn
import torch.nn.functional as F
class NetDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.do1 = nn.Dropout(0.2)  # 20% Probability
        self.fc1 = nn.Linear(250000, 120)
        self.do2 = nn.Dropout(0.2)  # 20% Probability
        self.fc2 = nn.Linear(120, 84)
        self.do3 = nn.Dropout(0.1)  # 10% Probability
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.do1(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.do2(x)
        x = F.relu(self.fc2(x))
        x = self.do3(x)
        x = self.fc3(x)
        return x

In [33]:

class AlexNet2(nn.Module):
    def __init__(self):
        super(AlexNet2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 6400, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 4096)
        self.fc3 = nn.Linear(in_features=4096 , out_features=2)


    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [34]:
AlexNet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


In [35]:
#Updating the second classifier
AlexNet.classifier[4] = nn.Linear(4096,1024)
AlexNet.classifier[6] = nn.Linear(1024,2)

In [38]:
def one_pass(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0

    if backwards:
    for x, y in tqdm(dataloader):

        y_pred = model(x)
        # print(y_pred)
        # print(type(y_pred))
        # print(y_pred.shape())
        # print()
        # print(y)
        # print(type(y))
        # print(y.shape())
        loss = lossFun(y_pred, y)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    else:
        for x, y in dataloader:
          
          y_pred = model(x)
          loss = lossFun(y_pred, y)
          total_loss += loss.item()


    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss

In [39]:
alexnet2 = AlexNet2()
alexnet2.to(device)

optimizer = optim.Adam(alexnet2.parameters(), lr = 0.001)
trained_alexnet, train_losses_alexnet, valid_losses_alexnet = model_train_eval(alexnet2, dl_train, optimizer, num_epochs=10)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/132 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train: 
- loss:  0.7451
- Accuracy:  72.64 %
- Precision:  72.64 %
- Recall:  100.0 %

Valid: 
- loss:  0.64832
- Accuracy:  62.86 %
- Precision:  62.86 %
- Recall:  100.0 %
Epoch:  1


...

Train: 
- loss:  0.58948
- Accuracy:  72.64 %
- Precision:  72.64 %
- Recall:  100.0 %

Valid: 
- loss:  0.66732
- Accuracy:  62.86 %
- Precision:  62.86 %
- Recall:  100.0 %
Epoch:  8


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.59181
- Accuracy:  72.64 %
- Precision:  72.64 %
- Recall:  100.0 %

Valid: 
- loss:  0.65387
- Accuracy:  62.86 %
- Precision:  62.86 %
- Recall:  100.0 %
Epoch:  9


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.58943
- Accuracy:  72.64 %
- Precision:  72.64 %
- Recall:  100.0 %

Valid: 
- loss:  0.70481
- Accuracy:  62.86 %
- Precision:  62.86 %
- Recall:  100.0 %


## ResNet

In [24]:
class ResNet(nn.Module):
    def __init__(self, config, output_dim):
        super().__init__()
                
        block, n_blocks, channels = config
        self.in_channels = channels[0]
            
        assert len(n_blocks) == len(channels) == 4
        
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size = 7, stride = 2, padding = 3, bias = False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self.get_resnet_layer(block, n_blocks[0], channels[0])
        self.layer2 = self.get_resnet_layer(block, n_blocks[1], channels[1], stride = 2)
        self.layer3 = self.get_resnet_layer(block, n_blocks[2], channels[2], stride = 2)
        self.layer4 = self.get_resnet_layer(block, n_blocks[3], channels[3], stride = 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(self.in_channels, output_dim)
        
    def get_resnet_layer(self, block, n_blocks, channels, stride = 1):
    
        layers = []
        
        if self.in_channels != block.expansion * channels:
            downsample = True
        else:
            downsample = False
        
        layers.append(block(self.in_channels, channels, stride, downsample))
        
        for i in range(1, n_blocks):
            layers.append(block(block.expansion * channels, channels))

        self.in_channels = block.expansion * channels
            
        return nn.Sequential(*layers)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.fc(h)
        
        return x, h

In [ ]:
resnet = models.resnet18(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)

resnet.to(device)

optimizer = optim.Adam(resnet.parameters(), lr = 0.001)
trained_alexnet, train_losses_alexnet, valid_losses_alexnet = model_train_eval(resnet, dl_train, optimizer, num_epochs=15)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/132 [00:00<?, ?it/s]

...

  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.10708
- Accuracy:  96.28 %
- Precision:  98.61 %
- Recall:  96.23 %

Valid: 
- loss:  0.30446
- Accuracy:  80.0 %
- Precision:  82.61 %
- Recall:  86.36 %
Epoch:  7


  0%|          | 0/132 [00:00<?, ?it/s]

Train: 
- loss:  0.11114
- Accuracy:  96.2 %
- Precision:  99.45 %
- Recall:  95.29 %

Valid: 
- loss:  0.08437
- Accuracy:  97.14 %
- Precision:  100.0 %
- Recall:  95.45 %
Epoch:  8


  0%|          | 0/132 [00:00<?, ?it/s]

In [ ]:
# resnet = models.resnet18(pretrained=True)
# num_ftrs = resnet.fc.in_features
# resnet.fc = nn.Linear(num_ftrs, 2)

# resnet.to(device)

# optimizer = optim.Adam(resnet.parameters(), lr = 0.001)
# trained_alexnet, train_losses_alexnet, valid_losses_alexnet = model_train_eval(resnet, dl_train, optimizer, num_epochs=10)




In [ ]:
lrs = [0.0001, 0.0005, 0.001, 0.005, 0.01]

In [ ]:
results_train = {}
results_test = {}
for lr in lrs:
    resnet = models.resnet18(pretrained=True)
    num_ftrs = resnet.fc.in_features
    resnet.fc = nn.Linear(num_ftrs, 2)

    resnet.to(device)


    optimizer = optim.Adam(resnet.parameters(), lr = lr)

    trained_alexnet, train_losses_alexnet, valid_losses_alexnet = model_train_eval(resnet, dl_train, optimizer, num_epochs=10)

    results_train[lr] = (train_losses_alexnet[-1])
    results_test[lr] = (valid_losses_alexnet[-1])

  
